<a href="https://colab.research.google.com/github/Leonardorochagit/Geracao_Energia/blob/main/Previsao_GeracaoEnergia_AM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Universidade do Estado do Amazonas</center>
###  <center>Escola Superior de Tecnologia</center>
#####  <center>Pós-graduação Lato Sensu em Ciência de Dados</center>
#####  <center>Programação para Ciência de Dados (Turma 02)</center>
---
## <center>TCC de Conclusão de Curso </center>
## <center> Previsão de Geração de Energia: Amazonas </center>


## **1. Importação das Bibliotecas e Configurações Ambiente**



In [ ]:
# !pip install sweetviz -q
# !pip install -U pandas-profiling
# !pip install --user -q datascience

In [ ]:
!pip install --user -q pandas -plotting

In [ ]:
! git clone https://github.com/Leonardorochagit/Geracao_Energia
#! git clone https://github.com/paulordie/sentimental_ds
%cd Geracao_Energia/Base_ONS/

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import pyplot
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline 

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.filters.hp_filter import hpfilter
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from datetime import datetime
import seaborn as sns
plt.rcParams["figure.figsize"] = (15,15)
import seaborn as sns

In [ ]:
# palette -> Accent, Accent_r, Blues, Blues_r, BrBG, BrBG_r, BuGn, BuGn_r, BuPu, BuPu_r, CMRmap, CMRmap_r, Dark2, Dark2_r, GnBu, GnBu_r, Greens, Greens_r, Greys, Greys_r, OrRd, OrRd_r, Oranges, Oranges_r, PRGn, PRGn_r, Paired, Paired_r, Pastel1, Pastel1_r, Pastel2, Pastel2_r, PiYG, PiYG_r, PuBu, PuBuGn, PuBuGn_r, PuBu_r, PuOr, PuOr_r, PuRd, PuRd_r, Purples, Purples_r, RdBu, RdBu_r, RdGy, RdGy_r, RdPu, RdPu_r, RdYlBu, RdYlBu_r, RdYlGn, RdYlGn_r, Reds, Reds_r, Set1, Set1_r, Set2, Set2_r, Set3, Set3_r, Spectral, Spectral_r, Wistia, Wistia_r, YlGn, YlGnBu, YlGnBu_r, YlGn_r, YlOrBr, YlOrBr_r, YlOrRd, YlOrRd_r, afmhot, afmhot_r, autumn, autumn_r, binary, binary_r, bone, bone_r, brg, brg_r, bwr, bwr_r, cividis, cividis_r, cool, cool_r, coolwarm, coolwarm_r, copper, copper_r, cubehelix, cubehelix_r, flag, flag_r, gist_earth, gist_earth_r, gist_gray, gist_gray_r, gist_heat, gist_heat_r, gist_ncar, gist_ncar_r, gist_rainbow, gist_rainbow_r, gist_stern, gist_stern_r, gist_yarg, gist_yarg_r, gnuplot, gnuplot2, gnuplot2_r, gnuplot_r, gray, gray_r, hot, hot_r, hsv, hsv_r, icefire, icefire_r, inferno, inferno_r, jet, jet_r, magma, magma_r, mako, mako_r, nipy_spectral, nipy_spectral_r, ocean, ocean_r, pink, pink_r, plasma, plasma_r, prism, prism_r, rainbow, rainbow_r, rocket, rocket_r, seismic, seismic_r, spring, spring_r, summer, summer_r, tab10, tab10_r, tab20, tab20_r, tab20b, tab20b_r, tab20c, tab20c_r, terrain, terrain_r, viridis, viridis_r, vlag, vlag_r, winter, winter_r
sns.set_palette("Blues_r")

# style -> white, dark, whitegrid, darkgrid, ticks
sns.set_style("darkgrid")

In [ ]:
pd.options.display.max_columns = None

## **2. Abrindo a Base de Dados** 

In [ ]:
# Convertendo o dataset em um dataframe com Pandas
df = pd.read_csv('/content/Geracao_Energia/Base_ONS/4. Simples_Geração_de_Energia_Dia_data AMAZONAS.csv', sep = ';', 
                 index_col=0, header=0, parse_dates=True, squeeze=True) # header=0, index_col=0, parse_dates=True, squeeze=True
df.head()    

In [ ]:
df.columns


In [ ]:
lista_colunas_drop = ['cod_aneel (tb_referenciacegusina (Usina))', 'cod_nucleoaneel (tb_referenciacegusina (Usina))', 
                      'Data Dica','dsc_estado', 'id_subsistema', 'nom_tipousinasite', 'nom_usina2', 
                      'Período Exibido GE']
df=df.drop(columns=lista_colunas_drop)
df

In [ ]:
df = df.rename(columns={'Data Escala de Tempo 1 GE Simp 4':'Data/Hora','Selecione Tipo de GE Simp 4':'Geracao_(GWh)'})
#df = df.reset_index(drop=True)
df.head(10)

In [ ]:
df.index.names = ['Data/Hora']
df

In [ ]:
df['Geracao_(GWh)']=df['Geracao_(GWh)'].apply(lambda x: str(x).replace(',','.'))
df

In [ ]:
df['Geracao_(GWh)']=df['Geracao_(GWh)'].astype('float64')

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.index

In [ ]:
df['year'] = df.index.year
df['month']  = df.index.month
df['weekday'] = df.index.weekday
df['weekofyear'] = df.index.weekofyear
df['quarter'] = df.index.quarter
df.tail(3)

## **3. Manipulando a Base de Dados** 

In [ ]:
df['Geracao_(GWh)'].plot()

In [ ]:
df.info()

In [ ]:
df1 = df.iloc[:,0:1]

In [ ]:
df1

In [ ]:
df1.info()

In [ ]:
# Grouping data based on month and store type
dfbox= df1.groupby([pd.Grouper(freq='A'), 'Geracao_(GWh)']).sum().head(15)

In [ ]:
dfbox.head(30)

In [ ]:
# from pandas import DataFrame
# years=DataFrame()
# for name, group in df1: 
#    years[name.year]=group.values
# years.boxplot
# pyplot.show()

In [ ]:
pd.date_range('2008-01-01', '2022-01-10', freq='M')

In [ ]:
# Specify the data columns we want to include (i.e. exclude Year, Month, Weekday Name)
data_columns = ['Geracao_(GWh)']
# Resample to weekly frequency, aggregating with mean
df_ano = df1[data_columns].resample('Y').sum()
df_ano.head(15)

In [ ]:
years = DataFrame()
for name, group in groups:
	years[name.year] = group.values
years.boxplot()
pyplot.show()

In [ ]:
df_ano['Geracao_(GWh)'].plot()

In [ ]:
df_ano

In [ ]:
sns.df_ano(x=df_ano.index, y='Geracao_(GWh)', data=df_ano)

In [ ]:
ts = df1, index=pd.date_range(start="2013-01-01", periods=n, freq="Y"))
fig, ax = plt.subplots(figsize=(12,5))
seaborn.boxplot(ts.index.dayofyear, ts, ax=ax)

In [ ]:
from pandas import read_csv
from pandas import DataFrame
from pandas import Grouper
from matplotlib import pyplot

In [ ]:
groups = df1.groupby(Grouper(freq='A'))
groups.head()

In [ ]:
#Try lag 1 day
pd.plotting.lag_plot(df['Geracao_(GWh)'], lag =24 )

In [ ]:
#Try lag 365 day
pd.plotting.lag_plot(df['Geracao_(GWh)'], lag = 8640)

Auto-correlation Plots

In [ ]:
from pandas.plotting import autocorrelation_plot

In [ ]:
autocorrelation_plot(df)
pyplot.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(df, lags=31)
pyplot.show()

Partial Auto-Correlation

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(df, lags=50)
pyplot.show()

## **3. Normalizando** #

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn.preprocessing
from sklearn.metrics import r2_score

from keras.layers import Dense,Dropout,SimpleRNN,LSTM
from keras.models import Sequential

#check all the files in the input dataset
#print(os.listdir("../input/"))

In [ ]:
df

In [ ]:
temp = df.copy() # make temporary copy of dataframe
dataset = temp['Geracao_(GWh)'].values # numpy.ndarray of the actual load
dataset = dataset.astype('float32') 
dataset = np.reshape(dataset, (-1, 1)) # reshape to one feature; required for the models

#scaler = MinMaxScaler(feature_range=(0, 1)) # Min Max scaler
#dataset = scaler.fit_transform(dataset) # fit and transform the dataset

# Train and Test splits
train_size = int(len(dataset) * 0.80) 
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)
    
look_back = 25 # timesteps to lookback for predictions
X_train, trainY = create_dataset(train, look_back)
X_test, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
print("Shapes: \nTraining set: {}, Testing set: {}".format(X_train.shape, X_test.shape))
print("Sample from training set: \n{}".format(X_train[0]))

##Models and their MAPE
Here we test various models and visualize their predictions. Models used are:

AutoRegressive
Moving Average
ARMA
ARIMA
LSTM

In [ ]:
from statsmodels.tsa.ar_model import AR

model = AR(train)
model_fit = model.fit()

##LSTM

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
# Any results you write to the current directory are saved as output.

In [ ]:
temp = df
dataset = temp['Geracao_(GWh)'].dropna().values #numpy.ndarray
dataset = dataset.astype('float32')
dataset = np.reshape(dataset, (-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1)) # Min Max scaler
dataset = scaler.fit_transform(dataset)
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)
    
look_back = 25
X_train, Y_train = create_dataset(train, look_back)
X_test, Y_test = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(X_train, Y_train, epochs=200, batch_size=70, validation_data=(X_test, Y_test),verbose=1, shuffle=False)

model.summary()

In [ ]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
# invert predictions
train_predict = scaler.inverse_transform(train_predict)
Y_train = scaler.inverse_transform([Y_train])
test_predict = scaler.inverse_transform(test_predict)
Y_test = scaler.inverse_transform([Y_test])
print('Train Mean Absolute Error:', mean_absolute_error(Y_train[0], train_predict[:,0]))
print('Train Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_train[0], train_predict[:,0])))
print('Test Mean Absolute Error:', mean_absolute_error(Y_test[0], test_predict[:,0]))
print('Test Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_test[0], test_predict[:,0])))

In [ ]:
mape_train = np.mean(np.abs((Y_train[0] - train_predict[:,0]) / Y_train[0])) * 100
mape_test = np.mean(np.abs((Y_test[0] - test_predict[:,0]) / Y_test[0])) * 100

print("Train MAPE: {}, Test MAPE: {}".format(mape_train, mape_test))

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show();

In [ ]:
idx = 200
aa=[x for x in range(idx)]
plt.figure(figsize=(8,4))
plt.plot(aa, Y_test[0][:idx], marker='.', label="actual")
plt.plot(aa, test_predict[:,0][:idx], 'r', label="prediction")
# plt.tick_params(left=False, labelleft=True) #remove ticks
plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('TOTAL Load', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();